# Import libraries

In [ ]:
import psycopg2
import pandas as pd
import random
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time

# Initialize Spotify client

In [ ]:
# Set your Spotify API credentials
client_id = '6c2ff081fe494968a2e3a618c678fd62'
client_secret = '99da4179a82f4f999058535d8d9b47dc'

# Initialize Spotify client
sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id, client_secret))

# Connect to DB

In [ ]:
# Connect to DB
db_params = {
    "host": "rain.db.elephantsql.com",
    "dbname": "auspovuc",  # Replace with your desired database name
    "user": "auspovuc",  # Replace with your PostgreSQL username
    "password": "bmJdG19Daw9rkEsJ3VnkefRGCBF_oy7F",  # Replace with your PostgreSQL password
}

conn = psycopg2.connect(**db_params)
cur = conn.cursor()

# Request user input

In [1]:
# Ask user's age_group
age = input('Please indicate your age group:\n  1.0-17 \n  2.18-34 \n  3.35-59 \n  4.60-74 \n  5.76 and above\nEnter a number: ')

Please indicate your age group:
  1.0-17 
  2.18-34 
  3.35-59 
  4.60-74 
  5.76 and above
Enter a number: 1


In [2]:
# Ask if user has any of the 4 conditions
feels = input('How are you feeling today (select a number)?\n  1.Anxious \n  2.Depressed \n  3.Can\'t sleep \n  4.I have OCD \n  5.I feel great!\nEnter a number: ')

How are you feeling today (select a number)?
  1.Anxious 
  2.Depressed 
  3.Can't sleep 
  4.I have OCD 
  5.I feel great!
Enter a number: 2


# Suggest track(s) according to input

In [ ]:
# Declare conditions
conditions = ['anxiety', 'depression', 'insomnia', 'ocd']

# Declare age_group
groups = ['early_years', 'young_adults', 'mature_adults', 'middle_age', 'elderly']

# Ensure print dataframe with full column width for url display
pd.set_option('display.max_colwidth', None)  

# look for top choice of genre based on user input (*SQL INJECTION RISK)
if feels in ['1', '2', '3', '4']:
    if age in ['1', '2', '3', '4', '5']:
        x = int(feels)
        y = int(age)
        # query recommended_tracks for genre from specified view
        cur.execute(f"""
                SELECT t.track_name, t.artist, t.album, t.genre, t.preview_url 
                FROM recommended_tracks t
                JOIN {groups[y-1]}_{conditions[x-1]}_genre a
                ON a.genre = t.genre
                WHERE a.row_n = 1 ;
                """)
        conn.commit()
        result = cur.fetchall()
        #convert result to a dataframe and print    
        df = pd.DataFrame(result, columns=['Song', 'Artist', 'Album', 'Genre', 'Preview URL'])
        print('Have a listen to these songs. They just might make you feel better!\n\n', df)
    else:
        print('Hmm, you\'re good, you don\'t need help.')
    
elif feels == '5':
    # Define seed genres
    seed_genres = ['pagode']
    # Get recommended tracks for each seed genre
    for seed_genre in seed_genres:
        recommended_tracks = sp.recommendations(seed_genres=[seed_genre], limit=3, country="SG")
        for track in recommended_tracks["tracks"]:
            track_data = {
                        "track_name": track["name"],
                        "artist": ", ".join(artist["name"] for artist in track["artists"]),
                        "album": track["album"]["name"],
                        "genre": seed_genre,
                        "preview_url": track["preview_url"]
                            }
        # Create a DataFrame from the track data, suggest random track from top 3
        random_track = {random.randint(0,3)}
        df = pd.DataFrame(track_data, index=[random_track])
        print('Check this out!\n', df)
else:
    print('Hmm, you\'re good, you don\'t need help.')

# Close connection

In [ ]:
# close connection to db
conn.close()